In [4]:
import pdfplumber
import pandas as pd
import re
import sys

from module.read_payslip import pdfReader, payslipReader

In [6]:
vng_payslip_reader = payslipReader(r"D:\Hungtv7\Personal_Fin\VNG_payslip\payslip_VG-15316_2021_04.pdf")

# tables = vng_payslip_reader.extract_tables() # Extract tables from the PDF file
transposed_tables = vng_payslip_reader.clean_tables() # Clean the tables
vng_payslip_reader.exportColName(transposed_tables) # Export column names to a JSON file